# Rename the SmartSolo export files into SeisComP Data Structure (SDS) format

In [2]:
import os, sys
import numpy as np
import obspy
import datetime
import pandas as pd
import shutil
from tqdm import tqdm

## Specify input folder

In [71]:
path_input = 'E:/Nodes_ablation_zone/All_comp_DP_/'
path_output = 'E:/rename_minions'

if os.path.exists(path_output) == False :
    os.mkdir(path_output)

## Read files in input folder

In [72]:
files = np.array(os.listdir(path_input))

### Extracting all the station name

In [73]:
sp = np.chararray.split(files, '.')

nL = []
for nn in sp :
    if nn[-1] == 'miniseed' : 
        nL.append(nn[0])
    
SttName = np.unique(np.array(nL))

print(SttName)

['453020739' '453020754' '453020769' '453020773' '453020774' '453020781']


### Determining the NET STA LOC CHAN TYPE YEAR DAY for each files

In [74]:
info = []

for i, file in enumerate(files) :
    if (file[:9] in SttName) == True :
        if i == 0 or (file[:9] != files[i-1][:9]) :
            St = obspy.read(path_input+file)
            sta = St[0].stats.station
            net = St[0].stats.network
            print(net, sta)
        
        year = int(file[-34:-30])
        day = datetime.datetime.strptime(file[-34:-24], '%Y.%m.%d') - datetime.datetime.strptime(file[-34:-30], '%Y')
        day = day.days + 1
        chan = 'DP' + file[-10:-9]
        Type = 'D' 
        loc = ''
        
        info.append([net,sta,loc,chan,Type,year,day, file])

SS 20739
SS 20754
SS 20769
SS 20773
SS 20774
SS 20781


In [75]:
headers = ['Network', 'Station', 'Location','Channel', 'Type', 'Year','Day','File']
file_info = pd.DataFrame(np.array(info), columns=headers)

In [76]:
file_info

,Network,Station,Location,Channel,Type,Year,Day,File
0,SS,20739,,DPE,D,2021,301,453020739.1.2021.10.28.00.50.46.000.E.miniseed
1,SS,20739,,DPN,D,2021,301,453020739.1.2021.10.28.00.50.46.000.N.miniseed
2,SS,20739,,DPZ,D,2021,301,453020739.1.2021.10.28.00.50.46.000.Z.miniseed
3,SS,20739,,DPE,D,2022,135,453020739.10.2022.05.15.00.00.00.000.E.miniseed
4,SS,20739,,DPN,D,2022,135,453020739.10.2022.05.15.00.00.00.000.N.miniseed
...,...,...,...,...,...,...,...,...
544,SS,20781,,DPN,D,2022,133,453020781.8.2022.05.13.00.00.00.000.N.miniseed
545,SS,20781,,DPZ,D,2022,133,453020781.8.2022.05.13.00.00.00.000.Z.miniseed
546,SS,20781,,DPE,D,2022,134,453020781.9.2022.05.14.00.00.00.000.E.miniseed
547,SS,20781,,DPN,D,2022,134,453020781.9.2022.05.14.00.00.00.000.N.miniseed


## Convert and organize minions file into SDS format

In [113]:
for daInfo in tqdm(info, total=len(info)) :
    
    net = daInfo[0]
    sta = daInfo[1]
    loc = daInfo[2]
    chan = daInfo[3]
    Type = daInfo[4]
    year = daInfo[5]
    day = daInfo[6]
    file = daInfo[-1]
    
    file_path = '%s/%s/%s/%s/%s.%s/' %(path_output,str(year),str(net),str(sta),str(chan),Type)
    
    if os.path.exists(file_path) == False :
        os.makedirs(file_path)
    
    output_file = '%s.%s.%s.%s.%s.%s.%s' %(net, sta, loc,chan,Type,year,day)
    dst = file_path + output_file
    shutil.copy2(path_input+file, dst)

100%|████████████████████████████████████████████████████████████████████████████████| 549/549 [02:34<00:00,  3.55it/s]
